In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:35763")
client

<Client: 'tcp://127.0.0.1:35763' processes=8 threads=64, memory=755.46 GiB>

In [2]:
import xarray
import colorcet as cc
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib
import numpy as np
import os
import pandas as pd
import json
import seaborn
import yaml
import copy
from pyeosim.atmosphere import LUT_from_file
from pyeosim.datasets import DATA_PATHS
from pyeosim.spectral import TreeView_1, TreeView_2, Sentinel2VNIR, SuperDove, TreeView_3
from pyeosim.sensor import TCMOS_test, TeledyneCMOS
from pyeosim.plot import rgb
from hsman import get_datasets, open_dataset


In [12]:
ds = open_dataset('OUMI1202006221_VNIR_aerial',
                  chunks={'band':200,'y':500, 'x':500}).isel(y=slice(7000,14500),
                                                     x=slice(10000,17500))

surface_reflectance = ds.swap_dims({'band':'wavelength'})
# slice between 400 and 898 to reduce data volume and align with the bandpass responses used later in simulation
# Also rescale to reflectance by divide by 10000
surface_reflectance = surface_reflectance.sel(wavelength=slice(400,898)).astype(float)/10000
# replace anything with a greater than 1 reflectance by truncating at 1
surface_reflectance = surface_reflectance.where(surface_reflectance <= 1, other=1).compute()

In [13]:
surface_reflectance

<xarray.DataArray 'reflectance' (wavelength: 153, y: 7500, x: 7500)>
array([[[0.0776, 0.0716, 0.0824, ..., 0.0273, 0.026 , 0.024 ],
        [0.0748, 0.0784, 0.0774, ..., 0.0257, 0.0226, 0.0256],
        [0.0695, 0.0668, 0.0628, ..., 0.0225, 0.0257, 0.0261],
        ...,
        [0.0372, 0.0378, 0.0389, ..., 0.0203, 0.0238, 0.0191],
        [0.0361, 0.0364, 0.0359, ..., 0.0211, 0.0209, 0.0193],
        [0.0361, 0.0376, 0.0325, ..., 0.0236, 0.021 , 0.0203]],

       [[0.077 , 0.0717, 0.0815, ..., 0.0271, 0.0259, 0.0241],
        [0.0742, 0.0776, 0.0767, ..., 0.0248, 0.0227, 0.0255],
        [0.0687, 0.0669, 0.0618, ..., 0.0223, 0.0251, 0.0263],
        ...,
        [0.037 , 0.0381, 0.039 , ..., 0.021 , 0.0234, 0.0193],
        [0.0365, 0.0374, 0.0363, ..., 0.0209, 0.0214, 0.0198],
        [0.0365, 0.0385, 0.0339, ..., 0.0234, 0.0216, 0.0207]],

       [[0.0766, 0.0717, 0.0803, ..., 0.0268, 0.0262, 0.0245],
        [0.0732, 0.077 , 0.0761, ..., 0.0248, 0.0225, 0.0254],
        [0.0672, 0.0665, 0.061 , ..., 0.0223, 0.0248, 0.0265],
        ...,
...
        ...,
        [0.2609, 0.2618, 0.2656, ..., 0.5388, 0.5452, 0.5383],
        [0.2669, 0.2732, 0.2611, ..., 0.5299, 0.5383, 0.5128],
        [0.2669, 0.2586, 0.2499, ..., 0.5218, 0.503 , 0.4801]],

       [[0.1251, 0.1224, 0.1178, ..., 0.2601, 0.2736, 0.2625],
        [0.1258, 0.1142, 0.1118, ..., 0.2798, 0.2957, 0.2611],
        [0.1218, 0.1223, 0.1169, ..., 0.274 , 0.237 , 0.2439],
        ...,
        [0.2628, 0.2642, 0.2685, ..., 0.5373, 0.5465, 0.537 ],
        [0.2676, 0.2718, 0.2611, ..., 0.5306, 0.5384, 0.5109],
        [0.2676, 0.2609, 0.2519, ..., 0.5232, 0.503 , 0.4807]],

       [[0.1249, 0.1228, 0.1177, ..., 0.2617, 0.278 , 0.2626],
        [0.1265, 0.114 , 0.111 , ..., 0.2797, 0.2998, 0.2634],
        [0.122 , 0.1217, 0.1167, ..., 0.2751, 0.2396, 0.2447],
        ...,
        [0.2648, 0.2669, 0.271 , ..., 0.5367, 0.55  , 0.5376],
        [0.2691, 0.2715, 0.2608, ..., 0.5306, 0.5388, 0.5103],
        [0.2691, 0.2602, 0.2524, ..., 0.5233, 0.5019, 0.4793]]])
Coordinates:
    band        (wavelength) int64 1 2 3 4 5 6 7 ... 147 148 149 150 151 152 153
  * y           (y) float64 5.765e+06 5.765e+06 ... 5.763e+06 5.763e+06
  * x           (x) float64 6.517e+05 6.517e+05 ... 6.542e+05 6.542e+05
  * wavelength  (wavelength) float64 410.0 413.0 416.0 ... 890.0 893.0 896.0

In [14]:
# PARAMETERS
ATMOSPHERE_PATH = os.path.join('atmospheres', 'atmospheres_2')
ATMOSPHERE_SCENARIO = 'mk_20200622_1030_nadir'
spectral_response = TreeView_3()

In [15]:
def read_atmos_meta(path):
    """
    Loads JSON file containing atmospheric simulation parameters used in 6SV
    """
    jsons = os.listdir(path)
    jsons = [x for x in os.listdir(path) if x.endswith('json')]
    metas = {}
    for _json in jsons:
        with open(os.path.join(path, _json), 'r') as f:
            metas[_json.split('.')[0]] = json.load(f)
    return pd.DataFrame(metas).T

atmos_meta = read_atmos_meta(ATMOSPHERE_PATH)
atmos_meta

,version,month,day,solar_z,solar_a,view_z,view_a,scattering_angle,azimuthal_angle_difference,visibility,...,atmospheric_intrinsic_reflectance,background_reflectance,pixel_reflectance,direct_solar_irradiance,diffuse_solar_irradiance,environmental_irradiance,atmospheric_intrinsic_radiance,background_radiance,pixel_radiance,solar_spectrum
inv_20200622_1030_nadir,1.1,6,22,36,150,0,0,143.36,150.3,8.49,...,0.089,0.384,0.537,639.951,601.212,306.191,41.59,180.168,252.17,1836.922
inv_20200622_1200_nadir,1.1,6,22,34,186,0,0,145.86,186.04,8.49,...,0.091,0.387,0.541,677.159,611.508,317.91,44.036,187.063,261.821,1836.922
inv_20200301_1030_nadir,1.1,3,1,67,156,0,0,112.95,156.93,8.49,...,0.125,0.309,0.432,135.397,374.7,125.839,29.904,74.046,103.638,1933.95
inv_20201001_1030_nadir,1.1,10,1,62,162,0,0,117.81,162.28,8.49,...,0.112,0.33,0.461,210.29,427.994,157.463,31.415,92.654,129.682,1892.893
inv_20200801_1030_nadir,1.1,8,1,42,151,0,0,137.72,151.28,8.49,...,0.088,0.377,0.528,551.236,575.304,277.914,38.208,163.529,228.882,1841.392
inv_20200501_1030_nadir,1.1,5,1,44,155,0,0,135.81,155.35,8.49,...,0.088,0.374,0.524,527.183,573.474,271.529,37.746,159.772,223.623,1870.334
mk_20200622_1030_nadir,1.1,6,22,33,140,0,0,146.71,140.71,8.49,...,0.091,0.387,0.542,689.322,614.763,321.714,44.657,189.301,264.954,1836.922
mk_20200622_1200_nadir,1.1,6,22,28,180,0,0,151.4,180.45,8.49,...,0.092,0.391,0.547,752.294,630.802,341.206,47.228,200.771,281.007,1836.922
mk_20200301_1030_nadir,1.1,3,1,63,152,0,0,116.99,152.18,8.49,...,0.113,0.326,0.457,200.697,427.417,154.954,31.679,91.177,127.615,1933.95
mk_20201001_1030_nadir,1.1,10,1,57,157,0,0,122.21,157.28,8.49,...,0.103,0.345,0.482,287.355,474.943,188.056,33.019,110.655,154.878,1892.893


In [16]:
atmos = LUT_from_file(ATMOSPHERE_PATH, {
    'view_z': 0,
    'view_a': 0
})

In [17]:
atmos.LUT = atmos.LUT.sel(scenario=ATMOSPHERE_SCENARIO)

In [ ]:
# # generate Top of Atmosphere radiance array
radiance = atmos.transform(surface_reflectance)

In [ ]:
sensor_configs = yaml.load(open('sensors.yaml', 'r'), Loader=yaml.Loader)
print('Available Sensor Configurations:')
list(sensor_configs.keys())

In [ ]:
msi_sensor = TeledyneCMOS(**sensor_configs['TreeView_3_MSI_32'])
msi_sensor_simulation = msi_sensor.fit_transform(radiance)